In [4]:
import subprocess

In [50]:
import os
import subprocess

def get_input(dayNum):
    data_file = f"./data/day-{dayNum}.input"
    if not os.path.exists(data_file):
        download_input(dayNum) 
    return [line.strip() for line in open(data_file).readlines()]

def download_input(dayNum):
    SESSION_COOKIE = open('./session_cookie.secret').readlines()[0].strip()
    url = f"https://adventofcode.com/2020/day/{dayNum}/input"
    cmd = f"curl -H 'Cookie: session={SESSION_COOKIE}' {url} > ./data/day-{dayNum}.input"
    subprocess.run(cmd, capture_output=True, shell=True)